In [2]:
import statsmodels.stats.api as sms
from statsmodels.stats.power import TTestIndPower

In [8]:
##################################
# Power analysis to determine
# sample size
##################################

analysis = TTestIndPower()

# reject H0 if below this threshold (CI of .95) 
alpha = .05

# min change in values that is meaningful. We w ant to go from 50% to 55%. 
effect = sms.proportion_effectsize(.5,.55)

# power = probability this test will correctly reject H0. 0.8 is the convention.
power = .8

# Send these as parameters to solver_power(). Ratio concerns the relative sizes of the groups. We here keep them the same (1)
result = analysis.solve_power(effect, power=power, nobs1=None,
                              ratio=1.0, alpha=alpha) 
# 3 decimal places
print('%.3f' % result)

1565.490


In [12]:
##################################
# Import data
##################################
import pandas as pd
import numpy as np
import math
import scipy.stats as st
import matplotlib as mpl
import matplotlib.pyplot as plt

df = pd.read_csv('LSE_DA301_Week_1_files/Data/new_bike_shop_AB.csv')
print(df)
df.shape

        Unnamed: 0  RecordID     IPAddress  LoyaltyPage  ServerID  \
0                0         1   39.13.114.2            1         2   
1                1         2     13.3.25.8            1         1   
2                2         3   247.8.211.8            1         1   
3                3         4   124.8.220.3            0         3   
4                4         5   60.10.192.7            0         2   
...            ...       ...           ...          ...       ...   
184583      184583    184584   114.8.104.1            0         1   
184584      184584    184585   207.2.110.5            0         2   
184585      184585    184586   170.13.31.9            0         2   
184586      184586    184587   195.14.92.3            0         3   
184587      184587    184588  172.12.115.8            0         2   

        VisitPageFlag  
0                   0  
1                   0  
2                   0  
3                   0  
4                   0  
...               ...  
184

(184588, 6)

In [23]:
##################################
# Clean data
##################################
df_new = df.drop_duplicates()
df_new = df_new.drop(['Unnamed: 0', 'RecordID', 'VisitPageFlag'], axis=1)

In [25]:
df_new

,IPAddress,LoyaltyPage,ServerID
0,39.13.114.2,1,2
1,13.3.25.8,1,1
2,247.8.211.8,1,1
3,124.8.220.3,0,3
4,60.10.192.7,0,2
...,...,...,...
184583,114.8.104.1,0,1
184584,207.2.110.5,0,2
184585,170.13.31.9,0,2
184586,195.14.92.3,0,3


In [28]:
##################################
# Subset 
##################################
df_new['ServerID'] = df_new['ServerID'].map({1: 'Treatment', 2: 'Control', 3: 'Control'})
df_new.head()

,IPAddress,LoyaltyPage,ServerID
0,39.13.114.2,1,Control
1,13.3.25.8,1,Treatment
2,247.8.211.8,1,Treatment
3,124.8.220.3,0,Control
4,60.10.192.7,0,Control


In [29]:
# Splitting the df into the two groups.
# Then use the sample() method to sample the number we establish above i.e. 1565
control_sample = df_new[df_new['ServerID'] == 'Control'].sample(n=1565,
                                                 random_state=22) 

treatment_sample = df_new[df_new['ServerID'] == 'Treatment'].sample(n=1565,
                                                 random_state=22) 
print(treatment_sample)

           IPAddress  LoyaltyPage   ServerID
50986     172.13.4.7            0  Treatment
109572   198.5.215.5            1  Treatment
105604   16.15.101.7            0  Treatment
16478      3.4.151.8            1  Treatment
153531    3.12.162.3            0  Treatment
...              ...          ...        ...
145353    38.0.192.8            1  Treatment
171688   63.12.126.1            1  Treatment
134780   25.15.142.4            0  Treatment
126747  138.11.162.1            0  Treatment
106900     198.2.9.3            1  Treatment

[1565 rows x 3 columns]


In [40]:
##################################
# Perform A/B test 
##################################

# Put them in same df
ab_test = pd.concat([control_sample, treatment_sample], axis=0)
ab_test.reset_index(drop=True, inplace=True)

# groupby based on Control or Treatment
conversion_rates = ab_test.groupby('ServerID')['LoyaltyPage']

# Calculate the mean, standard deviation and standard error of the mean

conversion_rates = conversion_rates.agg([np.mean, np.std, st.sem])
conversion_rates.columns = ['conversion_rate', 'STD_p','SE_p']

# 3 decimal places
conversion_rates.style.format('{:.3f}')

,conversion_rate,STD_p,SE_p
ServerID,,,
Control,0.506,0.500,0.013
Treatment,0.500,0.500,0.013


In [41]:
# Calculate p-value and confidence intervals
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
control_results = ab_test[ab_test['ServerID']=='Control']['LoyaltyPage']
treatment_results = ab_test[ab_test['ServerID']=='Treatment']['LoyaltyPage']

# Number of samples
n_con = control_results.count()
n_treat = treatment_results.count()

# Number of conversions 
successes = [control_results.sum(), treatment_results.sum()]

# (Put sample counts them in a list) 
nobs = [n_con, n_treat]

# Calculate the stats we want (z-test stat, p-value, CI for each group)

# z-stat and p-value
z_stat, pval = proportions_ztest(successes, nobs=nobs)
# CI
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes,
                                                                        nobs=nobs,
                                                                        
                                                                        alpha=0.05)
print(f'Z test stat: {z_stat:.2f}')
print(f'P-value: {pval:.3f}')
print(f'Confidence Interval of 95% for control group: [{lower_con:.3f}, {upper_con:.3f}]')
print(f'Confidence Interval of 95% for treatment group: [{lower_treat:.3f}, {upper_treat:.3f}]')

Z test stat: 0.32
P-value: 0.748
Confidence Interval of 95% for control group: [0.481, 0.531]
Confidence Interval of 95% for treatment group: [0.476, 0.525]


# Summary
We saw that the control conversion rate was slightly higher than for the the treatment group (i.e. .506 vs .50). 
In any case, the p=value is > .05. Therefore, we CANNOT reject H0, which is that the new page design performs better than the old design. In other words, given the samples we chose, the differences between control and treatment conversions are consitent with H0. 